In [184]:
import os
import pandas as pd

In [185]:
def get_data_file_path(current_trial_name="v2_eval_20_3", base_folder="game_logs", trial_base_name="alfworld_eval_", csv_file_name="alfworld_results.csv"):
    """returns the csv file path."""
    current_trial_folder = trial_base_name + current_trial_name
    file_path = os.path.join(base_folder, current_trial_folder, csv_file_name)
    return file_path

In [186]:
def convert_string_to_bool(in_string):
    """if string ="" False, else True"""
    if in_string:
        return True
    else:
        return False

def convert_string_to_bool_int(in_string):
    """if string ="" False, else True"""
    if in_string:
        return 1
    else:
        return 0

def convert_bool_string_to_bool_int(in_string):
    """ if string==False -> False, etc."""
    if in_string=="(False,)" or in_string=="False":
        return 0
    elif in_string=="(True,)" or in_string=="True":
        return 1
    else:
        print(type(in_string))
        print(in_string)
        print(in_string[0])
        raise Exception("Error in Transforming BoolString to Int")

def modify_text_prompt(keys_removed):
    """changes to short version"""
    keys = keys_removed.count("+")
    if keys==0:
        return keys_removed
    elif keys>1:
        return "short"
    else:
        return "long"

In [285]:
def load_data(curren_trial_name, filter_flag=False):
    """Loads the data"""
    file_path = get_data_file_path(current_trial_name=curren_trial_name)
    results_data = pd.read_csv(
        os.path.join("..",file_path), 
        converters = {
            "early_stop":convert_string_to_bool_int, 
            "error": convert_string_to_bool_int,
            "done": convert_bool_string_to_bool_int,
            "success": convert_bool_string_to_bool_int,
            "keys_removed": modify_text_prompt
        }
    )
    header_order = ["model", "keys_removed","temperature","env_type","success","done","early_stop","num_of_steps","num_nothing_happens","num_repetitions","error", "env_idx"]
    clean_results = results_data[header_order]

    # Filters for short, long and react-2 prompts
    if filter_flag: 
        filtering_condition = (clean_results["keys_removed"]=="short") | (clean_results["keys_removed"]=="long") | (clean_results["keys_removed"]=="react-2") | (clean_results["keys_removed"]=="react-1")| (clean_results["keys_removed"]=="long-2") | (clean_results["keys_removed"]=="short-2") | (clean_results["keys_removed"]=="short-2-swaped")
        clean_results = clean_results[ filtering_condition ]
    
    return clean_results

In [286]:
FLAG_ALL = False
FLAG_ALL = True

if FLAG_ALL:
    CURRENT_TRIAL_NAME = "v2_eval_20_3"
    CURRENT_TRIAL_NAME2 = "v2_eval_20-40"
    CURRENT_TRIAL_NAME3 = "v2_eval_60-40"
    CURRENT_TRIAL_NAME4 = "v2_eval_100-35"
    CURRENT_TRIAL_NAME5 = "v2_eval_0-60"
    CURRENT_TRIAL_NAME6 = "v2_eval_60-75"
    CURRENT_TRIAL_NAME7 = "v2_eval_20-115"
    CURRENT_TRIAL_NAME8 = "v2_eval_0-135"


    
    clean_results1 = load_data(CURRENT_TRIAL_NAME, True)
    clean_results2 = load_data(CURRENT_TRIAL_NAME2)
    clean_results3 = load_data(CURRENT_TRIAL_NAME3)
    clean_results4 = load_data(CURRENT_TRIAL_NAME4)
    clean_results5 = load_data(CURRENT_TRIAL_NAME5)
    clean_results6 = load_data(CURRENT_TRIAL_NAME6)
    clean_results7 = load_data(CURRENT_TRIAL_NAME7)
    clean_results8 = load_data(CURRENT_TRIAL_NAME8)
   
    
    # clean_results5,clean_results6,
    clean_results = pd.concat([clean_results1,clean_results2,clean_results3, clean_results4,clean_results7,clean_results8])
    print(clean_results.shape)
else:
    CURRENT_TRIAL_NAME4 = "v2_eval_20_3"
    clean_results = load_data(CURRENT_TRIAL_NAME)

clean_results.head()

(1215, 12)


,model,keys_removed,temperature,env_type,success,done,early_stop,num_of_steps,num_nothing_happens,num_repetitions,error,env_idx
0,gpt-3.5-turbo-0125,react-1,0.0,cool,0,1,0,49,12,0,0,0
1,gpt-3.5-turbo-0125,react-1,0.0,clean,0,1,0,49,13,0,0,1
2,gpt-3.5-turbo-0125,react-1,0.0,clean,0,1,0,49,33,0,0,2
3,gpt-3.5-turbo-0125,react-1,0.0,put,0,1,0,49,26,0,0,3
4,gpt-3.5-turbo-0125,react-1,0.0,clean,0,1,0,49,39,2,0,4


In [287]:
# This is the table that shows results per env_type
out = clean_results.groupby(["model","keys_removed","env_type"]).sum().reset_index()
out["total"] = clean_results.groupby(["model","keys_removed","env_type"])["success"].count().reset_index()["success"]
out["success_std"] = clean_results.groupby(["model","keys_removed","env_type"])["success"].std().reset_index()["success"]
out["done_std"] = clean_results.groupby(["model","keys_removed","env_type"])["done"].std().reset_index()["done"]
out_header_order = ["model", "keys_removed","temperature","env_type","total","success","success_std","done","done_std","early_stop","num_of_steps","num_nothing_happens","num_repetitions"]
out = out[out_header_order]
# Adding a few more scores.
out["normalised_nothing"] = out["num_nothing_happens"] / out["num_of_steps"]
out["normalised_reps"] = out["num_repetitions"] / out["num_of_steps"]
out["success_percentage"] = out["success"] / out["total"]
out["success_percentage_done"] = out["success"] / out["done"]
out["done_percentage"] = out["done"] / out["total"]
out["normalised_total"] = out["total"] / 135
out["normalised_done"] = out["done"] / out["total"]

x = out.groupby(["env_type"])
x.head()

,model,keys_removed,temperature,env_type,total,success,success_std,done,done_std,early_stop,num_of_steps,num_nothing_happens,num_repetitions,normalised_nothing,normalised_reps,success_percentage,success_percentage_done,done_percentage,normalised_total,normalised_done
0,gpt-3.5-turbo-0125,long,0.0,clean,31,17,0.505879,25,0.401610,6,735,325,46,0.442177,0.062585,0.548387,0.680000,0.806452,0.229630,0.806452
1,gpt-3.5-turbo-0125,long,0.0,cool,21,2,0.300793,11,0.511766,10,751,296,83,0.394141,0.110519,0.095238,0.181818,0.523810,0.155556,0.523810
2,gpt-3.5-turbo-0125,long,0.0,examine,18,5,0.460889,13,0.460889,5,553,290,54,0.524412,0.097649,0.277778,0.384615,0.722222,0.133333,0.722222
3,gpt-3.5-turbo-0125,long,0.0,heat,23,13,0.506870,20,0.344350,3,521,270,35,0.518234,0.067179,0.565217,0.650000,0.869565,0.170370,0.869565
4,gpt-3.5-turbo-0125,long,0.0,put,25,5,0.408248,14,0.506623,11,760,379,84,0.498684,0.110526,0.200000,0.357143,0.560000,0.185185,0.560000
5,gpt-3.5-turbo-0125,long,0.0,puttwo,17,1,0.242536,8,0.514496,9,644,264,82,0.409938,0.127329,0.058824,0.125000,0.470588,0.125926,0.470588
6,gpt-3.5-turbo-0125,long-2,0.0,clean,31,16,0.508001,22,0.461414,9,690,221,60,0.320290,0.086957,0.516129,0.727273,0.709677,0.229630,0.709677
7,gpt-3.5-turbo-0125,long-2,0.0,cool,21,8,0.497613,15,0.462910,6,634,278,53,0.438486,0.083596,0.380952,0.533333,0.714286,0.155556,0.714286
8,gpt-3.5-turbo-0125,long-2,0.0,examine,18,1,0.235702,7,0.501631,11,504,469,133,0.930556,0.263889,0.055556,0.142857,0.388889,0.133333,0.388889
9,gpt-3.5-turbo-0125,long-2,0.0,heat,23,8,0.486985,11,0.510754,12,450,185,67,0.411111,0.148889,0.347826,0.727273,0.478261,0.170370,0.478261


In [288]:
out.to_csv("../results/pandas_output_per_env_type.csv",sep="&",index=False)

In [290]:
print(out[["model", "keys_removed","env_type","total","success"]])

                 model    keys_removed env_type  total  success
0   gpt-3.5-turbo-0125            long    clean     31       17
1   gpt-3.5-turbo-0125            long     cool     21        2
2   gpt-3.5-turbo-0125            long  examine     18        5
3   gpt-3.5-turbo-0125            long     heat     23       13
4   gpt-3.5-turbo-0125            long      put     25        5
5   gpt-3.5-turbo-0125            long   puttwo     17        1
6   gpt-3.5-turbo-0125          long-2    clean     31       16
7   gpt-3.5-turbo-0125          long-2     cool     21        8
8   gpt-3.5-turbo-0125          long-2  examine     18        1
9   gpt-3.5-turbo-0125          long-2     heat     23        8
10  gpt-3.5-turbo-0125          long-2      put     25       12
11  gpt-3.5-turbo-0125          long-2   puttwo     17        4
12  gpt-3.5-turbo-0125   long-2-swaped    clean     31       12
13  gpt-3.5-turbo-0125   long-2-swaped     cool     21       10
14  gpt-3.5-turbo-0125   long-2-swaped  

In [291]:
def create_pivot(variable="success"):# pivot1 = out.pivot(index=["keys_removed"],columns=["env_type"], values=["success"])
    pivot_table = pd.pivot_table(out, index=["model","keys_removed"],columns=["env_type"], aggfunc={variable:"first"})
    pivot_table.columns = pivot_table.columns.droplevel(0)
    # pivot1.columns.name=None
    piv_table = pivot_table.reset_index()
    return piv_table

piv1 = create_pivot("success")
piv1.head()

env_type,model,keys_removed,clean,cool,examine,heat,put,puttwo
0,gpt-3.5-turbo-0125,long,17,2,5,13,5,1
1,gpt-3.5-turbo-0125,long-2,16,8,1,8,12,4
2,gpt-3.5-turbo-0125,long-2-swaped,12,10,6,9,16,2
3,gpt-3.5-turbo-0125,react-1,1,0,7,7,1,0
4,gpt-3.5-turbo-0125,react-2,7,1,12,13,3,7


In [292]:
pivot_done = create_pivot("done")
pivot_done.head()

env_type,model,keys_removed,clean,cool,examine,heat,put,puttwo
0,gpt-3.5-turbo-0125,long,25,11,13,20,14,8
1,gpt-3.5-turbo-0125,long-2,22,15,7,11,14,8
2,gpt-3.5-turbo-0125,long-2-swaped,18,18,9,11,18,12
3,gpt-3.5-turbo-0125,react-1,28,20,14,23,25,17
4,gpt-3.5-turbo-0125,react-2,27,16,14,20,24,14


In [293]:
pivot_done = create_pivot("num_repetitions")
pivot_done.head()

env_type,model,keys_removed,clean,cool,examine,heat,put,puttwo
0,gpt-3.5-turbo-0125,long,46,83,54,35,84,82
1,gpt-3.5-turbo-0125,long-2,60,53,133,67,68,62
2,gpt-3.5-turbo-0125,long-2-swaped,88,26,73,80,41,59
3,gpt-3.5-turbo-0125,react-1,26,16,33,7,13,3
4,gpt-3.5-turbo-0125,react-2,43,41,27,37,16,22


In [294]:
pivot_done = create_pivot("num_nothing_happens")
pivot_done.head()

env_type,model,keys_removed,clean,cool,examine,heat,put,puttwo
0,gpt-3.5-turbo-0125,long,325,296,290,270,379,264
1,gpt-3.5-turbo-0125,long-2,221,278,469,185,214,184
2,gpt-3.5-turbo-0125,long-2-swaped,273,203,250,273,130,279
3,gpt-3.5-turbo-0125,react-1,750,541,273,365,609,359
4,gpt-3.5-turbo-0125,react-2,479,486,135,265,520,175


In [296]:
pivot_done = create_pivot("success_percentage") #higher is better
# pivot_done.head()
pivot_done

env_type,model,keys_removed,clean,cool,examine,heat,put,puttwo
0,gpt-3.5-turbo-0125,long,0.548387,0.095238,0.277778,0.565217,0.20,0.058824
1,gpt-3.5-turbo-0125,long-2,0.516129,0.380952,0.055556,0.347826,0.48,0.235294
2,gpt-3.5-turbo-0125,long-2-swaped,0.387097,0.476190,0.333333,0.391304,0.64,0.117647
3,gpt-3.5-turbo-0125,react-1,0.032258,0.000000,0.388889,0.304348,0.04,0.000000
4,gpt-3.5-turbo-0125,react-2,0.225806,0.047619,0.666667,0.565217,0.12,0.411765
5,gpt-3.5-turbo-0125,react-2-swaped,0.258065,0.333333,0.777778,0.608696,0.16,0.470588
6,gpt-3.5-turbo-0125,short,0.516129,0.142857,0.388889,0.478261,0.28,0.000000
7,gpt-3.5-turbo-0125,short-2,0.483871,0.380952,0.222222,0.478261,0.32,0.176471
8,gpt-3.5-turbo-0125,short-2-swaped,0.580645,0.571429,0.166667,0.478261,0.68,0.176471


In [297]:
pivot_done = create_pivot("normalised_nothing") #lower is better (this shows how many of the actions result in "nothing happens"
pivot_done.head()

env_type,model,keys_removed,clean,cool,examine,heat,put,puttwo
0,gpt-3.5-turbo-0125,long,0.442177,0.394141,0.524412,0.518234,0.498684,0.409938
1,gpt-3.5-turbo-0125,long-2,0.320290,0.438486,0.930556,0.411111,0.422091,0.362919
2,gpt-3.5-turbo-0125,long-2-swaped,0.399707,0.364452,0.577367,0.542744,0.298165,0.423369
3,gpt-3.5-turbo-0125,react-1,0.530035,0.537239,0.473958,0.433492,0.514793,0.430972
4,gpt-3.5-turbo-0125,react-2,0.402183,0.526544,0.402985,0.433007,0.471869,0.281804


In [298]:
pivot_done = create_pivot("normalised_done") #higher is better (this shows how many reached done, without early stop)
pivot_done.head()

env_type,model,keys_removed,clean,cool,examine,heat,put,puttwo
0,gpt-3.5-turbo-0125,long,0.806452,0.523810,0.722222,0.869565,0.56,0.470588
1,gpt-3.5-turbo-0125,long-2,0.709677,0.714286,0.388889,0.478261,0.56,0.470588
2,gpt-3.5-turbo-0125,long-2-swaped,0.580645,0.857143,0.500000,0.478261,0.72,0.705882
3,gpt-3.5-turbo-0125,react-1,0.903226,0.952381,0.777778,1.000000,1.00,1.000000
4,gpt-3.5-turbo-0125,react-2,0.870968,0.761905,0.777778,0.869565,0.96,0.823529


In [299]:
pivot_done = create_pivot("success_percentage_done") #higher is better (this show successful run out of all done [not total]
pivot_done.head()

env_type,model,keys_removed,clean,cool,examine,heat,put,puttwo
0,gpt-3.5-turbo-0125,long,0.680000,0.181818,0.384615,0.650000,0.357143,0.125000
1,gpt-3.5-turbo-0125,long-2,0.727273,0.533333,0.142857,0.727273,0.857143,0.500000
2,gpt-3.5-turbo-0125,long-2-swaped,0.666667,0.555556,0.666667,0.818182,0.888889,0.166667
3,gpt-3.5-turbo-0125,react-1,0.035714,0.000000,0.500000,0.304348,0.040000,0.000000
4,gpt-3.5-turbo-0125,react-2,0.259259,0.062500,0.857143,0.650000,0.125000,0.500000


In [300]:
pivot_done = create_pivot("normalised_reps") #lower is better (this shows number of reps out of all steps, mind you we cut after 5 exact reps)
pivot_done.head()

env_type,model,keys_removed,clean,cool,examine,heat,put,puttwo
0,gpt-3.5-turbo-0125,long,0.062585,0.110519,0.097649,0.067179,0.110526,0.127329
1,gpt-3.5-turbo-0125,long-2,0.086957,0.083596,0.263889,0.148889,0.134122,0.122288
2,gpt-3.5-turbo-0125,long-2-swaped,0.128843,0.046679,0.168591,0.159046,0.094037,0.089530
3,gpt-3.5-turbo-0125,react-1,0.018375,0.015889,0.057292,0.008314,0.010989,0.003601
4,gpt-3.5-turbo-0125,react-2,0.036104,0.044420,0.080597,0.060458,0.014519,0.035427


In [301]:
pivot2 = pd.pivot_table(out, columns=["env_type"], aggfunc={"total":"first"})
pivot2["keys_removed"] = "total"
piv2 = pivot2.reset_index()
piv2.head()

env_type,index,clean,cool,examine,heat,put,puttwo,keys_removed
0,total,31,21,18,23,25,17,total


In [302]:
pivot3 = pd.pivot_table(out, columns=["env_type"], aggfunc={"normalised_total":"first"})
pivot3["keys_removed"] = "total"
piv3 = pivot3.reset_index()
piv3.head()

env_type,index,clean,cool,examine,heat,put,puttwo,keys_removed
0,normalised_total,0.22963,0.155556,0.133333,0.17037,0.185185,0.125926,total


In [303]:
new = pd.concat([piv1,piv2])
new = new.drop("index",axis=1)
new.head()

env_type,model,keys_removed,clean,cool,examine,heat,put,puttwo
0,gpt-3.5-turbo-0125,long,17,2,5,13,5,1
1,gpt-3.5-turbo-0125,long-2,16,8,1,8,12,4
2,gpt-3.5-turbo-0125,long-2-swaped,12,10,6,9,16,2
3,gpt-3.5-turbo-0125,react-1,1,0,7,7,1,0
4,gpt-3.5-turbo-0125,react-2,7,1,12,13,3,7


In [304]:
# This is the table that shows total results.
out = clean_results.groupby(["model","keys_removed"]).sum().reset_index()
out["total"] = clean_results.groupby(["model","keys_removed"])["success"].count().reset_index()["success"]
out_header_order = ["model", "keys_removed","temperature","total","success","done","early_stop","num_of_steps","num_nothing_happens","num_repetitions"]
out = out[out_header_order]
out["normalised_nothing"] = out["num_nothing_happens"] / out["num_of_steps"]
out["normalised_reps"] = out["num_repetitions"] / out["num_of_steps"]
out["success_percentage"] = out["success"] / out["total"]
out["success_percentage_done"] = out["success"] / out["done"]
out["done_percentage"] = out["done"] / out["total"]
x = out.groupby("keys_removed")
x.head()

,model,keys_removed,temperature,total,success,done,early_stop,num_of_steps,num_nothing_happens,num_repetitions,normalised_nothing,normalised_reps,success_percentage,success_percentage_done,done_percentage
0,gpt-3.5-turbo-0125,long,0.0,135,43,91,44,3964,1824,384,0.460141,0.096872,0.318519,0.472527,0.674074
1,gpt-3.5-turbo-0125,long-2,0.0,135,49,77,58,3292,1551,443,0.471142,0.134569,0.362963,0.636364,0.570370
2,gpt-3.5-turbo-0125,long-2-swaped,0.0,135,55,86,47,3271,1408,367,0.430449,0.112198,0.407407,0.639535,0.637037
3,gpt-3.5-turbo-0125,react-1,0.0,135,16,127,8,5856,2897,98,0.494706,0.016735,0.118519,0.125984,0.940741
4,gpt-3.5-turbo-0125,react-2,0.0,135,43,115,20,4784,2060,186,0.430602,0.038880,0.318519,0.373913,0.851852
5,gpt-3.5-turbo-0125,react-2-swaped,0.0,135,55,120,15,4380,1812,166,0.413699,0.037900,0.407407,0.458333,0.888889
6,gpt-3.5-turbo-0125,short,0.0,135,44,78,57,3691,1841,393,0.498781,0.106475,0.325926,0.564103,0.577778
7,gpt-3.5-turbo-0125,short-2,0.0,135,49,88,47,3425,1455,350,0.424818,0.102190,0.362963,0.556818,0.651852
8,gpt-3.5-turbo-0125,short-2-swaped,0.0,135,64,98,37,3274,1582,354,0.483201,0.108125,0.474074,0.653061,0.725926


In [145]:
summary_header = ["keys_removed","success_percentage","success_percentage_done","done_percentage","normalised_reps","normalised_nothing"]
outsummary = out[summary_header]
x = outsummary.groupby("keys_removed")
x.head()

,keys_removed,success_percentage,success_percentage_done,done_percentage,normalised_reps,normalised_nothing
0,long,0.318519,0.472527,0.674074,0.096872,0.460141
1,long-2,0.362963,0.636364,0.570370,0.134569,0.471142
2,react-2,0.318519,0.373913,0.851852,0.038880,0.430602
3,short,0.325926,0.564103,0.577778,0.106475,0.498781
4,short-2,0.362963,0.556818,0.651852,0.102190,0.424818


In [137]:
out.to_csv("../results/pandas_output_all_results.csv",sep="&",index=False)

In [138]:
# This is the table that shows results per env_id
out = clean_results.groupby(["model","env_idx","keys_removed"]).sum().reset_index()
out_header_order = ["model", "keys_removed","temperature","env_idx","success","done","early_stop","num_of_steps","num_nothing_happens","num_repetitions"]
out = out[out_header_order]
x = out.groupby("env_idx")
x.head()

,model,keys_removed,temperature,env_idx,success,done,early_stop,num_of_steps,num_nothing_happens,num_repetitions
0,gpt-3.5-turbo-0125,long,0.0,0,0,0,1,33,21,7
1,gpt-3.5-turbo-0125,long-2,0.0,0,0,1,0,49,43,0
2,gpt-3.5-turbo-0125,react-2,0.0,0,1,1,0,9,0,0
3,gpt-3.5-turbo-0125,short,0.0,0,0,0,1,15,10,5
4,gpt-3.5-turbo-0125,short-2,0.0,0,1,1,0,13,4,0
...,...,...,...,...,...,...,...,...,...,...
670,gpt-3.5-turbo-0125,long,0.0,134,0,0,1,43,12,5
671,gpt-3.5-turbo-0125,long-2,0.0,134,1,1,0,7,1,0
672,gpt-3.5-turbo-0125,react-2,0.0,134,0,1,0,49,21,0
673,gpt-3.5-turbo-0125,short,0.0,134,0,1,0,49,29,0


In [139]:
out.pivot(index=["env_idx"],columns=["keys_removed"], values=["success"])

success                             
keys_removed    long long-2 react-2 short short-2
env_idx                                          
0                  0      0       1     0       1
1                  0      1       0     1       1
2                  1      1       0     0       1
3                  0      0       0     1       0
4                  0      0       0     0       0
...              ...    ...     ...   ...     ...
130                1      0       0     1       0
131                0      0       1     1       1
132                0      0       0     0       0
133                0      0       1     0       0
134                0      1       0     0       1

[135 rows x 5 columns]

In [140]:
x= pd.Series([0.22963,0.155556,0.133333,0.17037,0.185185,0.125926])
x

0    0.229630
1    0.155556
2    0.133333
3    0.170370
4    0.185185
5    0.125926
dtype: float64

In [141]:
x*20

0    4.59260
1    3.11112
2    2.66666
3    3.40740
4    3.70370
5    2.51852
dtype: float64